# Allen atlas image regression

In [3]:
import numpy as np
import os
import functools
import time
from pathlib import Path
from tqdm import tqdm
from allensdk.core.reference_space import ReferenceSpace
from allensdk.core.reference_space_cache import ReferenceSpaceCache
from atlas_functions import *
import ants
import nrrd
import scipy
import cv2
from PIL import Image
from EasyROI import EasyROI
from plantcv import plantcv as pcv


<class 'NoneType'>
[DEBUG] Welcome to easyROI
[DEBUG] Entered draw_polygon
[DEBUG] Draw 1 polygon(s)
[DEBUG] Draw multiple lines by dragging the cursor
[DEBUG] Double Click to complete the polygon
[DEBUG] Press Esc to leave the process


AttributeError: 'NoneType' object has no attribute 'copy'

In [4]:
#directory = search_for_file_path()
rsp, tree = open_AllenSDK()
isocortex_map, id_name_dict, bregma = map_generator(rsp, tree, structure='Isocortex')

#sma_mask = create_mask(isocortex_map, 656)
#contour = create_contour(isocortex_map)



NameError: name 'open_AllenSDK' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(isocortex_map,vmax=1300, cmap='gray', alpha=0.5)
plt.imshow(contour, cmap='binary_r', alpha=0.5)
plt.show()

# Saving isocortex Mask

In [ ]:
isocortex_mask = np.zeros(isocortex_map.shape)
mask_list = []
for i in id_name_dict:
    mask_list.append(i)
isocortex_mask = create_mask(isocortex_map, mask_list)
isocortex_mask_int8 = isocortex_mask.astype('uint8')

In [ ]:
plt.imshow(isocortex_mask)
plt.imshow()
plt.show()

In [ ]:
#filePath = search_for_file_path()
#image_name = 'isocortex_mask_int8'
#imageio.imwrite(str(filePath) + image_name + '.tif', isocortex_mask)

In [ ]:
#contour2 = create_contour(isocortex_map)
#contour.shape
#print(contour)
#print(type(contour))
#print(contour.dtype)
#contour = contour.astype('uint8')
#print(contour.dtype)

#path = search_for_file_path()
#name = 'contour'
#im = Image.fromarray(contour)
#im.save(os.path.join(path, name + ".tif"))

def npy_to_tif(data, name, path = 'search'):
    if path == 'search':
        path = search_for_file_path()
    else:
        pass
    data = data.astype('uint8')
    print(data.dtype)
    im = Image.fromarray(data)
    im.save(os.path.join(path, name + ".tif"))
    return

In [ ]:
contour2 = create_contour(isocortex_map)
print(contour2.dtype)
npy_to_tif(contour2, 'test')

In [ ]:
tif_to_nrrd("contour.tif")

In [ ]:
#im = Image.fromarray(isocortex_mask_int8)
#im.save(os.path.join(filePath, image_name + ".tif"))

# Opening both masks and finding transform

In [ ]:
#atlas_array = cv2.imread("/Users/vincentchouinard/Documents/GitHub/Atlas/saved_images/isocortex_mask.tif")
#atlas_array = cv2.imread("/Users/vincentchouinard/Documents/GitHub/Atlas/saved_images/isocortex_churchland_mask_int8.tif")
#atlas_array = atlas_array[:,:,0].astype('uint8')
#print(atlas_array.dtype)
#print(atlas_array[200,200])
#print(atlas_array[3,7])

In [ ]:
#atlas_array = atlas_array/255
#atlas_array = atlas_array.astype("uint8")
#plt.imshow(atlas_array, cmap="binary_r")
#plt.colorbar()
#plt.show()
#print(atlas_array.dtype)
#print(atlas_array[200,200])
#print(atlas_array[3,7])

In [ ]:
name = 'contour'
im = Image.fromarray(contour)
im.save(os.path.join(path, name + ".tif"))

In [ ]:
contour_ants = ants.image_read(directory + "/saved_images/contour.nrrd")

In [ ]:
#img = cv2.imread(directory + "/" + "saved_images/isocortex_mask_churchland_int8.tif", cv2.IMREAD_GRAYSCALE)

In [ ]:
def tif_to_nrrd(filename, path):
    img = cv2.imread(path + "/" + filename, cv2.IMREAD_GRAYSCALE)
    filename = Path(filename).stem
    nrrd.write(path + "/" + filename + '.nrrd', img)
    return filename + '.nrrd'

In [ ]:
# Opens both files in the directory I choose
tif_to_nrrd("isocortex_churchland_mask_v3.tif")
#tif_to_nrrd("isocortex_mask_int8.tif")

In [ ]:
#churchland_nrrd, header = nrrd.read(directory + "/saved_images/isocortex_mask_churchland_int8.nrrd")
#print(header)
#header['space directions'] = np.delete(header['space directions'], 2, 1)
#header['space directions'] = np.array([[30, 0],[0,30]])
#print(header)

In [ ]:
churchland = ants.image_read(directory + "/saved_images/isocortex_churchland_mask_v3.nrrd")
atlas = ants.image_read(directory + "/saved_images/isocortex_mask_int8.nrrd")

In [ ]:
#churchland = ants.image_read(directory + "/saved_images/rot_isocortex_mask_churchland_int8.nrrd")

In [ ]:
#atlas = ants.image_read(directory + "/saved_images/isocortex_mask_int8.nrrd")
contour_ants = ants.image_read("contour.nrrd")

In [ ]:
mytx = ants.registration(fixed=churchland, moving=atlas, type_of_transform = ('SyN'), syn_sampling = 8)

In [ ]:
#print(mytx)

In [ ]:
atlas_warped = ants.apply_transforms(churchland, atlas, mytx['fwdtransforms']).numpy()
contours_warped = ants.apply_transforms(churchland, contour_ants, mytx['fwdtransforms']).numpy()

In [ ]:
path = search_for_file_path()
filename = 'isocortex_churchland.tif'
img = cv2.imread(path + "/" + filename, cv2.IMREAD_GRAYSCALE)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
#plt.imshow(atlas_warped, alpha=0.5)
plt.imshow(contours_warped, cmap='binary_r', alpha=0.8)
#plt.imshow(churchland.numpy(), alpha=0.5)
plt.imshow(img, cmap='binary_r', alpha=0.6)

In [ ]:
registrationPath = search_for_file_path()

#fixedname = ''
#fixed = search_for_file_path()
#fixed = fixed + '/' fixedname


In [ ]:
def registration(registrationPath, folderName, fixed_filename, moving_filename, trsfm = 'SyN', sampling = 8, show='off'):
    #Creates new folder at registrationPath with name folderName. If folder already existant, closes function.
    newFolder = registrationPath + "/" + folderName
    
    if os.path.exists(newFolder):
        print(f'{folderName} already exists.')
        if identify_files(newFolder, fixed_filename) == [fixed_filename]:
            tif_to_nrrd(fixed_filename, newFolder)
            tif_to_nrrd(moving_filename, newFolder)
        else:
            print('Insert .tif files in folder before registering.')
            return
    
    else:
        print(f'Creating a folder named {folderName} in {registrationPath}')
        os.mkdir(newFolder)
        
        #Opens fixed and moving images
        print('Opening fixed .tif image')
        fixed = search_for_file_path()
        fixedImg = cv2.imread(fixed + "/" + fixed_filename, cv2.IMREAD_GRAYSCALE)
        cv2.imwrite(newFolder + '/' + fixed_filename, fixedImg)
        nrrd_fixedname = tif_to_nrrd(fixed_filename, newFolder)
    
    
        print('Opening moving .tif image')
        moving = search_for_file_path()
        movingImg = cv2.imread(moving + "/" + moving_filename, cv2.IMREAD_GRAYSCALE)
        cv2.imwrite(newFolder + '/' + moving_filename, movingImg)
        nrrd_movingname = tif_to_nrrd(moving_filename, newFolder)
    
    # .tif and .nrrd created.
    
    ants_fixedImg = ants.image_read(newFolder + '/' + nrrd_fixedname)
    ants_movingImg = ants.image_read(newFolder + '/' + nrrd_movingname)
    mytx = ants.registration(fixed=ants_fixedImg, moving=ants_movingImg, type_of_transform = trsfm, syn_sampling = sampling)
    warpedImg = ants.apply_transforms(ants_fixedImg, ants_movingImg, mytx['fwdtransforms']).numpy()
    
    if show == 'on':
        fig, ax = plt.subplots(figsize=(10, 10))
        plt.imshow(warpedImg, cmap='PiYG', alpha=0.5)
        plt.imshow(fixedImg, cmap='PuOr', alpha=0.5)
    
    return mytx, ants_fixedImg, ants_movingImg, warpedImg

In [ ]:
trsfm, fixed, moving, warped = registration(registrationPath, 'churchland_contours', 'isocortex_churchland_mask_v3.tif', 'isocortex_mask_int8.tif', show='on')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(warped, cmap='PiYG', alpha=0.5)
plt.imshow(fixed.numpy(), cmap='binary_r', alpha=0.5)
#plt.imshow(moving.numpy(), cmap='binary_r', alpha=0.5)
plt.show()
    

In [2]:
path = search_for_file_path()
filename = 'isocortex_churchland.tif'
img_raw = cv2.imread(path + "/" + filename, cv2.IMREAD_GRAYSCALE)
roi_helper = EasyROI(verbose=True)
polygon_roi = roi_helper.draw_polygon(img_raw, 1)
contour, hierarchy = pcv.roi.custom(img=img_raw, vertices=polygon_roi["roi"][0]["vertices"])
mask = pcv.roi.roi2mask(img=img_raw, contour=contour)

You chose: /Users/vincentchouinard/Desktop
[DEBUG] Welcome to easyROI
[DEBUG] Entered draw_polygon
[DEBUG] Draw 1 polygon(s)
[DEBUG] Draw multiple lines by dragging the cursor
[DEBUG] Double Click to complete the polygon
[DEBUG] Press Esc to leave the process
[DEBUG] Not all ROI's drawn


KeyError: 'roi'